In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
from keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
print(os.listdir("../input"))

In [ ]:
import zipfile
localzip='/kaggle/input/dogs-vs-cats/train.zip'
zip_ref=zipfile.ZipFile(localzip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()
basedir = '/tmp/train'
imgnames = os.listdir(os.path.join(basedir))
imgnames[:10]

In [ ]:
sample = random.choice(imgnames)
image = load_img("/tmp/train/"+sample)
plt.imshow(image)
plt.show()

In [ ]:
filenames = os.listdir("/tmp/train/")
label = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        label.append(1)
    else:
        label.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'label': label
})
df.head()

In [ ]:
df['label'].value_counts().plot.bar(color = ['red','y'])

In [ ]:
df.shape

In [ ]:
df["label"] = df["label"].replace({0: 'cat', 1: 'dog'}) 

In [ ]:
train,test=train_test_split(df,test_size=0.20,random_state=42)
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)
train_sum=train.shape[0]
test_sum=test.shape[0]
batch_size=15

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
print(train.shape)
train['label'].value_counts().plot.bar(color=['r','b'])

In [ ]:
print(test.shape)
test['label'].value_counts().plot.bar(color=['r','b'])

# blance data set equal no of dogs and qual no cats

model building to the data sets

In [ ]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25) , 
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25) ,
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25) ,
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25) ,
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5) ,
    
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
model.summary()

Training Generator with Image Augmentation


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
total_train = train.shape[0]
total_test = test.shape[0]
batch_size=15

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train, 
    '/tmp/train',
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='binary',
    batch_size=32
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    test, 
    "/tmp/train/", 
    X_col='Image',
    Y_col='Label',
    target_size = (150, 150),
    batch_size = 20
)

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch = np.ceil(20000/20),  # 20000 images = batch_size * steps
      epochs = 10,
      validation_data=validation_generator,
      validation_steps = np.ceil(5000/20),  # 5000 images = batch_size * steps
      verbose = 1)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']